# Imports

In [1]:
from sklearn import *

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import re
from sklearn import tree

# FOR PART 1
import geopandas as gpd

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV

# Data import (Part 1 and Part 2)


In [2]:
train = pd.read_csv("cases_2021_train.csv")
test = pd.read_csv("cases_2021_test.csv")
location = pd.read_csv("location_2021.csv")

# Visualization (Part 1)


In [3]:
# Inspired by the docs: https://geopandas.org/en/v0.9.0/docs/user_guide/mapping.html

def visualize(data):
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

  # we need to know the total size of the dataset, to know the proportion
  size = data.shape[0]

  # count occurences by country, turn this into a DF, and reset indices so country is a column
  grouped = data.groupby(by="country").size().to_frame(name="size").reset_index()

  # to account for the huge impalance of data towards india
  grouped["size"] = np.log(grouped["size"])

  grouped.rename(columns={"country": "name"}, inplace=True)
  merged = world.merge(grouped, how="left", on="name")
  merged["size"].fillna(0, inplace=True)

  fig, ax = plt.subplots(1, 1)
  merged.plot(column="size", ax=ax, cmap="seismic", legend=True, legend_kwds={"label": "log(# of data points by country)", "orientation": "horizontal"})

In [ ]:
visualize(train)

#Data Preprocessing (Part 2)


In [ ]:
modifiedLocation = location.rename(columns = {"Country_Region":"country", "Province_State":"province", "Lat":"latitude", "Long_":"longitude"})

# Inner join of location and cases
data = pd.merge(left = modifiedLocation, right = train, on = ["country", "province"], how = "inner", )

# Remove all unnecessary columns
data = data.drop(columns=["additional_information", "source","Last_Update", "Combined_Key", "longitude_x", "latitude_x"])



# Drop all the NAs in every column except "province"
trainData = data.dropna(subset=["sex", "age", "Confirmed", "Deaths", "Recovered", "Active", "Case_Fatality_Ratio", "Incident_Rate", "longitude_y", "latitude_y", "outcome", "chronic_disease_binary" ])


#Remove all duplicates
# trainData = data.drop_duplicates()
# trainData = data

# Prepare outcome_group column by mapping all outcomes to 3 outcome_groups (hospitalized, nonhospitalized, deceased)

hospitalizedList = ["discharge", "discharged", "stable","stable condition","Receiving Treatment", "Hospitalized"]
nonhospitalizedList = ["Alive", "recovered", "Recovered"]
deceasedList = ["death", "Deceased", "died"]

# write into outcome group
conditions = [
    trainData["outcome"].isin(hospitalizedList),
    trainData["outcome"].isin(nonhospitalizedList),
    trainData["outcome"].isin(deceasedList)
]

values = ["hospitalized", "nonhospitalized", "deceased"]

# use the mask to apply values
trainData["outcome_group"] = np.select(conditions, values, default="")

# Rename and put the columns in the order of reference dataset
trainData = trainData.rename(columns= {"latitude_y":"latitude", "longitude_y":"longitude"})
trainData = trainData[["age", "sex", "province", "country", "latitude", "longitude", "date_confirmation", "chronic_disease_binary", "Confirmed", "Deaths", "Recovered", "Active", "Incident_Rate", "Case_Fatality_Ratio", "outcome_group"]]
trainData = trainData.drop_duplicates()
trainData

# 3. Feature selection

## Load The Clean Provided Data

In [8]:
data = pd.read_excel("./cases_2021_train_processed.xlsx", parse_dates=["date_confirmation"])
unlabelled = pd.read_excel("./cases_2021_test_processed_unlabelled.xlsx", parse_dates=["date_confirmation"])

In [9]:
# drop unnecessary attributes
data.drop(["province", "date_confirmation", "country", "latitude", "longitude"], inplace=True, axis=1)
unlabelled.drop(["province", "date_confirmation", "country", "latitude", "longitude"], inplace=True, axis=1)

# 4. Mapping the features

In [10]:
# convert sex, chronic_disease_binary, outcome group
  # if date_confirmation included, maybe convert to timestamp, so distance is computable???

def map_data(data):
  columns = data.columns

  if "sex" in columns:
    # sex mapping: {"male": 1, "female": 0}
    data["sex"] = data["sex"].apply( lambda row: (row == "male")*1 )

  # chronic_disease_binary mapping
  if "chronic_disease_binary" in columns:
    data["chronic_disease_binary"] = data["chronic_disease_binary"].astype(int)

  # if "date_confirmation" in columns:
  #   data["date_confirmation"] = data["date_confirmation"].astype(np.int64)

  outcome_mapping = {"deceased": 0, "hospitalized": 1, "nonhospitalized": 2}
  # outcome group mapping
  if "outcome_group" in columns:
    data["outcome_group"] = data["outcome_group"].apply( lambda row: outcome_mapping.get(row, None) )
  return data

data = map_data(data)
unlabelled = map_data(unlabelled)
unlabelled

,age,sex,chronic_disease_binary,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio
0,59,0,0,747288,13297,603746,130245,681.949809,1.779368
1,79,1,0,886673,12719,858075,15879,1139.078325,1.434463
2,44,0,0,886673,12719,858075,15879,1139.078325,1.434463
3,36,1,0,886673,12719,858075,15879,1139.078325,1.434463
4,52,1,0,265527,1576,262371,1580,212.762145,0.593537
...,...,...,...,...,...,...,...,...,...
4299,66,1,0,997004,12567,956170,28267,1475.672533,1.260476
4300,66,1,0,747288,13297,603746,130245,681.949809,1.779368
4301,53,0,0,886673,12719,858075,15879,1139.078325,1.434463
4302,25,1,0,886673,12719,858075,15879,1139.078325,1.434463


In [11]:
print(data.iloc[0])

age                           18.000000
sex                            0.000000
chronic_disease_binary         0.000000
Confirmed                 265527.000000
Deaths                      1576.000000
Recovered                 262371.000000
Active                      1580.000000
Incident_Rate                212.762145
Case_Fatality_Ratio            0.593537
outcome_group                  1.000000
Name: 0, dtype: float64


# 5. Balancing the classes in the training dataset

In [ ]:
grouped = data.groupby(by="outcome_group")

print(grouped.size())

# sample each group by the size of the most under-represented group
min_sample_size = grouped.size().min()
balanced_data = grouped.sample(n=min_sample_size)

plt.hist(balanced_data["outcome_group"].to_numpy())

print()

# 6. Building Models


## Split the data


In [15]:
# X_train, X_test, y_train, y_test = train_test_split(balanced_data.iloc[:, :-1], balanced_data.iloc[:, -1], train_size=0.8)
balanced_data_X = balanced_data.iloc[:, :-1]
balanced_data_y = balanced_data.iloc[:, -1]

## Metric Function

In [16]:
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

# https://stackoverflow.com/questions/32401493/how-to-create-customize-your-own-scorer-function-in-scikit-learn

# define custom scorings

def deceased_f1_score(y_true, y_test):
  f1_scores = f1_score( y_true , y_test, average=None )
  deceased_f1_score = f1_scores[0]
  return deceased_f1_score

def mean_macro_f1_score(y_true, y_test):
  f1_scores = f1_score( y_true , y_test, average=None )
  return np.mean(f1_scores)

def accuracy(y_true, y_test):
  return np.sum(y_true == y_test) / y_true.shape[0]


deceased_f1_scorer = make_scorer(deceased_f1_score, greater_is_better=True)
mean_macro_f1_scorer = make_scorer(mean_macro_f1_score, greater_is_better=True)
accuracy_scorer = make_scorer(accuracy, greater_is_better=True)


## Grid Search, cross validation and training



In [17]:
from sklearn.model_selection import GridSearchCV
import sys
def grid_search(model, param_grid):
  GS = GridSearchCV(
      model,
      param_grid,
      refit="f1_macro",
      scoring={"f1_deceased": deceased_f1_scorer, "f1_macro": mean_macro_f1_scorer, "accuracy": accuracy_scorer},
      verbose = 3,
      return_train_score=True,
      n_jobs = -1
      )

  # with open(f"{model[0]}.txt", "w") as f:
  #   sys.stdout = f
  #   GS.fit(balanced_data_X, balanced_data_y)
  #   sys.stdout = sys.__stdout__

  # create the estimator
  GS.fit(balanced_data_X, balanced_data_y)
  model = GS.best_estimator_
  scores = pd.DataFrame( GS.cv_results_ )
  scores = scores[ ["mean_test_f1_deceased", "mean_test_f1_macro", "mean_test_accuracy", "mean_train_f1_deceased", "mean_train_f1_macro", "mean_train_accuracy"] ]
  print(scores.iloc[GS.best_index_])
  print("Best parameters: ", GS.best_params_)
  # return model, scores.iloc[GS.best_index_], f
  return model, scores.iloc[GS.best_index_]



In [18]:
##create a dataframe with accuracies
accuracies = pd.DataFrame(columns=["model","mean_test_f1_deceased", "mean_test_f1_macro", "mean_test_accuracy", "mean_train_f1_deceased", "mean_train_f1_macro", "mean_train_accuracy"])

## Model 1: **Random Forest** ✅


In [21]:
# https://stackoverflow.com/questions/43366561/use-sklearns-gridsearchcv-with-a-pipeline-preprocessing-just-once
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    "rf__n_estimators": [i for i in range(10, 21, 1)],
    # "rf__min_samples_leaf": [5,7,10, 20, 25, 30],
    # 'rf__min_samples_split': [i for i in range(20,40,1)]
}

model = Pipeline(
    [
        ("rf", RandomForestClassifier())
    ]
)

model = grid_search(model, param_grid)



Fitting 5 folds for each of 11 candidates, totalling 55 fits
mean_test_f1_deceased     0.695306
mean_test_f1_macro        0.781108
mean_test_accuracy        0.782014
mean_train_f1_deceased    0.783917
mean_train_f1_macro       0.851315
mean_train_accuracy       0.852307
Name: 4, dtype: float64
Best parameters:  {'rf__n_estimators': 14}


## Model 2: **Gradient Boosting Classifier** ✅


In [22]:
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    "boost_tree__min_samples_split": [i for i in range(6, 21, 2)],
    "boost_tree__min_samples_leaf": [i for i in range(4, 10, 2)]
}

model = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("boost_tree", GradientBoostingClassifier())
    ]
)

model= grid_search(model, param_grid)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
mean_test_f1_deceased     0.707124
mean_test_f1_macro        0.794069
mean_test_accuracy        0.796394
mean_train_f1_deceased    0.744444
mean_train_f1_macro       0.822470
mean_train_accuracy       0.824474
Name: 12, dtype: float64
Best parameters:  {'boost_tree__min_samples_leaf': 6, 'boost_tree__min_samples_split': 14}


## Model 3: **Neural Network** ✅

In [ ]:
from sklearn.neural_network import MLPClassifier

param_grid = {
    'model__hidden_layer_sizes': [(12, 12, 4),(8, 10, 3), (10, 15, 3), (10, 20 ,3),],  # (10, 3), (12, 3) , (15, 3)
    'model__activation': [ 'tanh'],
    'model__max_iter': [2000],
    # 'model__learning_rate': ['constant'],
    'model__learning_rate_init': [0.00075, 0.0005, 0.001, 0.002, 0.005, 0.0075],
    # 'model__alpha': [0.0001, 0.0005, 0.00075, 0.001, 0.0025, 0.005],
}

model = Pipeline([
    ("scaler", StandardScaler()),
    ("model", MLPClassifier(hidden_layer_sizes=(12,12,4), learning_rate="constant", max_iter=2000, activation="tanh", learning_rate_init = 0.005, alpha =  0.00075)) #{'model__activation': 'tanh', 'model__alpha': 0.00075, 'model__hidden_layer_sizes': (12, 12, 4), 'model__learning_rate': 'constant', 'model__learning_rate_init': 0.005, 'model__max_iter': 2000}
    ]
)

model, _, outfile = grid_search(model, param_grid)



## Model 4: **KNN** ✅

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# https://stackoverflow.com/questions/43366561/use-sklearns-gridsearchcv-with-a-pipeline-preprocessing-just-once

param_grid = {
    "knn__n_neighbors": [i for i in range(1, 30)]
}

model = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("knn", KNeighborsClassifier())
    ]
)
model= grid_search(model, param_grid)



In [ ]:
accuracies.to_csv("accuracies.csv")


# 9. Prediction on the test set

In [24]:
#As it was mentioned in part 8, the best model is Boosting Classifier
import csv

def create_submission_file(y_preds, file_name):
 with open(file_name, "w") as csvfile:
  wr = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
  wr.writerow(["Id", "Prediction"])
  for i, pred in enumerate(y_preds):
    wr.writerow([str(i), str(pred)])


x_pred = unlabelled[:-1]

y_preds = model[0].predict(x_pred)
create_submission_file(y_preds, "2.submission_Boosting_Classifier.csv")